In [17]:
import psycopg2
conn = psycopg2.connect(
    dbname="hr_db",
    user="admin",
    password="admin@123",
    host="localhost",
    port="5432"
)
print("✅ Connected!")


✅ Connected!


In [19]:
pip install psycopg2-binary


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 385 kB 5.2 MB/s eta 0:00:01
  Created wheel for psycopg2-binary: filename=psycopg2_binary-2.9.10-cp39-cp39-macosx_10_9_universal2.whl size=242407 sha256=c11eaa17b2a350b8149e80dc436a5a144625ca227aef5c3298e62bac7f5c1707
  Stored in directory: /Users/nikhilshinde/Library/Caches/pip/wheels/b3/78/4b/26baded4713ddbdca47cb9dcdab88aae8371bdcda44f9e07eb
Successfully built psycopg2-binary
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
import random
from datetime import datetime, timedelta

from faker import Faker
from dotenv import load_dotenv
from sqlalchemy import create_engine, Column, Integer, String, Date, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# ------------------ SETUP ------------------
load_dotenv()
faker = Faker()
Base = declarative_base()

DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# ✅ Connect without exposing URI
engine = create_engine(
    "postgresql+psycopg2://",
    connect_args={
        "host": DB_HOST,
        "port": DB_PORT,
        "dbname": DB_NAME,
        "user": DB_USER,
        "password": DB_PASSWORD
    }
)

Session = sessionmaker(bind=engine)
session = Session()

# ------------------ TABLE DEFINITIONS ------------------

class PersonInfo(Base):
    __tablename__ = 'people_info'
    person_id = Column(Integer, primary_key=True)
    ssn = Column(String(11))
    full_name = Column(String(255))
    birth_date = Column(Date)
    age = Column(Integer)
    death_date = Column(Date, nullable=True)
    address_line1 = Column(String(255))
    address_line2 = Column(String(255))
    address_line3 = Column(String(255))
    city = Column(String(100))
    county = Column(String(100))
    zip_code = Column(String(10))
    birth_sex = Column(String(1))
    current_sex = Column(String(1))
    status = Column(String(1))
    company_id = Column(Integer)

class MedicalStaff(Base):
    __tablename__ = 'medical_staff'
    staff_id = Column(Integer, primary_key=True)
    full_name = Column(String(255))
    specialty = Column(String(100))
    role = Column(String(50))
    department = Column(String(100))
    status = Column(String(1))
    company_id = Column(Integer)
    created_on = Column(DateTime)

class PatientProfile(Base):
    __tablename__ = 'patient_profile'
    patient_id = Column(Integer, primary_key=True)
    person_id = Column(Integer, ForeignKey('people_info.person_id'))
    full_name = Column(String(255))
    primary_physician_id = Column(Integer)
    guarantor_id = Column(Integer)
    active_status = Column(String(1))
    created_on = Column(DateTime)

class VisitDetails(Base):
    __tablename__ = 'visit_details'
    visit_id = Column(Integer, primary_key=True)
    patient_id = Column(Integer, ForeignKey('patient_profile.patient_id'))
    visit_number = Column(Integer)
    patient_name = Column(String(255))
    admission_time = Column(DateTime)
    discharge_time = Column(DateTime)
    admission_status = Column(String(1))
    length_of_stay = Column(Integer)
    care_type_id = Column(Integer)
    admitting_company_id = Column(Integer)
    facility_id = Column(Integer)
    bed_id = Column(Integer)
    insurance_id = Column(Integer)
    finance_class_id = Column(Integer)
    primary_physician_id = Column(Integer)
    attending_doctor_id = Column(Integer, ForeignKey('medical_staff.staff_id'))
    admit_diagnosis = Column(String(255))

class Disease(Base):
    __tablename__ = 'diseases'
    disease_id = Column(Integer, primary_key=True)
    code = Column(String(10))
    name = Column(String(255))
    description = Column(String(500))
    status = Column(String(1))
    created_on = Column(DateTime)

class Medication(Base):
    __tablename__ = 'medications'
    medication_id = Column(Integer, primary_key=True)
    brand_name = Column(String(255))
    generic_name = Column(String(255))
    medication_code = Column(String(20))
    type = Column(String(50))
    dosage_form = Column(String(50))
    price_per_unit = Column(Integer)
    status = Column(String(1))
    created_on = Column(DateTime)

# ------------------ TABLE CREATION ------------------

Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

# ------------------ DATA GENERATION FUNCTIONS ------------------

def generate_people(n):
    people = []
    for _ in range(n):
        dob = faker.date_of_birth(minimum_age=1, maximum_age=95)
        dod = dob + timedelta(days=random.randint(3650, 30000)) if random.random() < 0.05 else None
        age = (datetime.now().date() - dob).days // 365
        people.append(PersonInfo(
            ssn=faker.ssn(),
            full_name=faker.name(),
            birth_date=dob,
            age=age,
            death_date=dod,
            address_line1=faker.street_address(),
            address_line2=faker.secondary_address(),
            address_line3='',
            city=faker.city(),
            county=faker.city_suffix(),
            zip_code=faker.zipcode(),
            birth_sex=random.choice(['M', 'F', 'U']),
            current_sex=random.choice(['M', 'F', 'U']),
            status=random.choice(['A', 'I']),
            company_id=random.randint(1, 5)
        ))
    return people

def generate_doctors(n):
    return [
        MedicalStaff(
            full_name=faker.name(),
            specialty=random.choice(['Cardiology', 'Neurology', 'General Medicine', 'Pediatrics', 'Orthopedics']),
            role='Physician',
            department=random.choice(['ICU', 'ER', 'Surgery', 'OPD']),
            status=random.choice(['A', 'I']),
            company_id=random.randint(1, 5),
            created_on=faker.date_time_this_decade()
        ) for _ in range(n)
    ]

def generate_patients(people):
    return [
        PatientProfile(
            person_id=person.person_id,
            full_name=person.full_name,
            primary_physician_id=random.randint(1, 200),
            guarantor_id=random.randint(1, 100),
            active_status=random.choice(['A', 'I']),
            created_on=faker.date_time_this_decade()
        ) for person in people
    ]

def generate_visits(patients, doctors):
    visits = []
    for patient in patients:
        for _ in range(random.randint(1, 4)):
            adm_time = faker.date_time_between(start_date='-3y', end_date='now')
            dis_time = adm_time + timedelta(days=random.randint(1, 10))
            visits.append(VisitDetails(
                patient_id=patient.patient_id,
                visit_number=random.randint(1000, 9999),
                patient_name=patient.full_name,
                admission_time=adm_time,
                discharge_time=dis_time,
                admission_status=random.choice(['Y', 'N', 'T', 'I', 'C']),
                length_of_stay=(dis_time - adm_time).days,
                care_type_id=random.randint(1, 5),
                admitting_company_id=random.randint(1, 5),
                facility_id=random.randint(1, 10),
                bed_id=random.randint(1, 300),
                insurance_id=random.randint(1, 50),
                finance_class_id=random.randint(1, 20),
                primary_physician_id=patient.primary_physician_id,
                attending_doctor_id=random.choice(doctors).staff_id,
                admit_diagnosis=faker.sentence(nb_words=6)
            ))
    return visits

def generate_diseases(n):
    return [
        Disease(
            code=f"DX{random.randint(100, 999)}",
            name=faker.catch_phrase(),
            description=faker.text(max_nb_chars=200),
            status=random.choice(['A', 'I']),
            created_on=faker.date_time_this_decade()
        ) for _ in range(n)
    ]

def generate_medications(n):
    return [
        Medication(
            brand_name=faker.company() + " Pharma",
            generic_name=faker.word().capitalize(),
            medication_code=f"{random.randint(10000, 99999)}-{random.randint(100, 999)}",
            type=random.choice(['Tablet', 'Injection', 'Syrup', 'Ointment']),
            dosage_form=random.choice(['100mg', '250mg', '500mg', '5ml', '10ml']),
            price_per_unit=random.randint(10, 200),
            status=random.choice(['A', 'I']),
            created_on=faker.date_time_this_decade()
        ) for _ in range(n)
    ]

# ------------------ DATA CREATION ------------------

print("🚀 Generating and inserting data...")

people = generate_people(2000)
session.add_all(people)
session.commit()

people = session.query(PersonInfo).all()

doctors = generate_doctors(200)
session.add_all(doctors)
session.commit()
doctors = session.query(MedicalStaff).all()

patients = generate_patients(people)
session.add_all(patients)
session.commit()
patients = session.query(PatientProfile).all()

visits = generate_visits(patients, doctors)
session.add_all(visits)
session.commit()

diseases = generate_diseases(100)
session.add_all(diseases)

medications = generate_medications(300)
session.add_all(medications)

session.commit()

print(f"✅ Done: {len(people)} people, {len(doctors)} doctors, {len(patients)} patients, {len(visits)} visits, {len(diseases)} diseases, {len(medications)} medications.")


/var/folders/qm/t1ynxc_53yb5_bszll45m6rh0000gn/T/ipykernel_85164/3094628212.py:14: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


🚀 Generating and inserting data...
✅ Done: 2000 people, 200 doctors, 2000 patients, 5024 visits, 100 diseases, 300 medications.
